In [1]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [2]:
files = utils.files_in_order('poses_compressed/shoulderpress')
X_train_names, X_test_names = train_test_split(files, test_size=0.2, random_state=42)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)

In [3]:
def load_features(names):
    output1 = [] # List of upper arm torso angles
    output2 = [] # List of forearm upper arm angles
    for filename in names:
        ps = load_ps('poses_compressed/shoulderpress/'+filename)
        poses = ps.poses

        right_present = [1 for pose in poses 
                if pose.rshoulder.exists and pose.relbow.exists and pose.rwrist.exists]
        left_present = [1 for pose in poses
                if pose.lshoulder.exists and pose.lelbow.exists and pose.lwrist.exists]
        right_count = sum(right_present)
        left_count = sum(left_present)
        side = 'right' if right_count > left_count else 'left'

        if side == 'right':
            joints = [(pose.rshoulder, pose.relbow, pose.rwrist, pose.rhip, pose.neck) for pose in poses]
        else:
            joints = [(pose.lshoulder, pose.lelbow, pose.lwrist, pose.lhip, pose.neck) for pose in poses]

        # filter out data points where a part does not exist
        joints = [joint for joint in joints if all(part.exists for part in joint)]
        joints_ = np.array(joints)
        
        back_vec = np.array([(joint[4].x - joint[3].x, joint[4].y - joint[3].y) for joint in joints])[:,0]
        
        elbow = joints_[:, 1]
        elbow_x = np.array([joint.x for joint in elbow])
        neck = joints_[:, 4]
        neck_x = np.array([joint.x for joint in neck])
        
        if side == 'right':
            arm_vec = elbow_x - neck_x
        else:
            arm_vec = neck_x - elbow_x
                
        upper_arm_vecs = np.array([(joint[0].x - joint[1].x, joint[0].y - joint[1].y) for joint in joints])
        forearm_vecs = np.array([(joint[2].x - joint[1].x, joint[2].y - joint[1].y) for joint in joints])
        
        upper_arm_vecs = upper_arm_vecs / np.expand_dims(np.linalg.norm(upper_arm_vecs, axis=1), axis=1)
        forearm_vecs = forearm_vecs / np.expand_dims(np.linalg.norm(forearm_vecs, axis=1), axis=1)
        
        upper_arm_forearm_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_arm_vecs, forearm_vecs), axis=1), -1.0, 1.0)))
        upper_arm_forearm_angle_filtered = medfilt(medfilt(upper_arm_forearm_angle, 5), 5)
        
        output1.append(back_vec.tolist())
        output2.append(upper_arm_forearm_angle_filtered.tolist())
    return output1, output2

X_train_1, X_train_2 = load_features(X_train_names)
X_test_1, X_test_2 = load_features(X_test_names)
#features = load_features(files)

In [4]:
predictions = []
for example in range(len(X_test_names)):
    # Store the average distance to good and bad training examples
    f1_good, f1_bad, f2_good, f2_bad = [[] for i in range(4)]
    
    # Compare distance of current test example with all training examples
    for i in range(len(X_train_1)):
        dist1 = utils.DTWDistance(X_train_1[i], X_test_1[example])
        dist2 = utils.DTWDistance(X_train_2[i], X_test_2[example])
        if y_train[i]:
            f1_good.append(dist1)
            f2_good.append(dist2)
        else:
            f1_bad.append(dist1)
            f2_bad.append(dist2)
    good_score = np.mean(f1_good) + np.mean(f2_good)
    bad_score = np.mean(f1_bad) + np.mean(f2_bad)
    
    if good_score < bad_score:
        predictions.append(1)
    else:
        predictions.append(0)
print(classification_report(y_test, predictions, target_names=['correct', 'incorrect']))

             precision    recall  f1-score   support

    correct       0.20      0.33      0.25         3
  incorrect       0.33      0.20      0.25         5

avg / total       0.28      0.25      0.25         8

